
In this assignment we are going to make panorama of two photos. It is called image stitching and you can read about in on Wikipedia: https://en.wikipedia.org/wiki/Image_stitching.

Example photos are available at Task 1 and the result is available at Task 9.

Although there are multiple implementation of image stitching using different libraries (OpenCV, skimage, ...) you should implement it using only `numpy` and `matplotlib` for displaying images.

You should write your own implementation of projective transformation fitting (using `numpy`) and of RANSAC.

# Task 1:


Take two photos and un-distort it. There should be 
rather big overlap (60% is ok) between those photos. 

In my case I will use (undistorted):

![](zdjecie_lewe.jpg)

and

![](zdjecie_prawe.jpg)

# Task 2:

Write a program, that apply projection transformation to a given image and displays both original image and transformed image.

**What projective transformation does**

(https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.ProjectiveTransform)

For each homogeneous coordinate $𝐱=[𝑥,𝑦,1]^𝑇$, its target position is calculated by multiplying with the given matrix, 𝐻, to give 𝐻𝐱

```
[[a0 a1 a2]
 [b0 b1 b2]
 [c0 c1 1 ]].
```

E.g., to rotate by theta degrees clockwise, the matrix should be:

```
[[cos(theta) -sin(theta) 0]
 [sin(theta)  cos(theta) 0]
 [0            0         1]]
```

or, to translate x by 10 and y by 20:

```
[[1 0 10]
 [0 1 20]
 [0 0 1 ]].
```




In [1]:
# TODO: your code goes here

Example:

![alt text](transformed.png "Transformed")

# Task 3:

We are going to use least-square method in order to find good fitting projective transformation based on pairs of matching points at different photos.
As a warm up let's solve a simple example:

Using https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html fit a plane $z = ax + by + c$ to the following data points:

```
x y z
0 1 2.5
1 2 2.5
2 3 4
3 4 6
```
<details>
<summary>Hint</summary>
my result is [0.4 , 1.05, 0.65]
</details>

In [ ]:
# your code goes here

# Task 4:

Using `linalg.lstsq` write a program that finds a projective transformation based on sequence of matching point coordinates.

The projective transformation is defined as:

```
X = (a0*x + a1*y + a2) / (c0*x + c1*y + 1)
Y = (b0*x + b1*y + b2) / (c0*x + c1*y + 1)
```

These equations can be transformed to the following form:

```
0 = a0*x + a1*y + a2 - c0*x*X - c1*y*X - X
0 = b0*x + b1*y + b2 - c0*x*Y - c1*y*Y - Y
```

which exist for each set of corresponding points, so we have a set of N * 2 equations. The coefficients appear linearly so we can write A x = 0, where:

```
A   = [[x y 1 0 0 0 -x*X -y*X -X]
       [0 0 0 x y 1 -x*Y -y*Y -Y]
        ...
        ...
      ]
x.T = [a0 a1 a2 b0 b1 b2 c0 c1 c3]
```

In case of total least-squares the solution of this homogeneous system of equations is the right singular vector of A which corresponds to the smallest singular value normed by the coefficient c3.

Weights can be applied to each pair of corresponding points to indicate, particularly in an overdetermined system, if point pairs have higher or lower confidence or uncertainties associated with them. From the matrix treatment of least squares problems, these weight values are normalised, square-rooted, then built into a diagonal matrix, by which A is multiplied.

In case of the affine transformation the coefficients c0 and c1 are 0. Thus the system of equations is:

```
A   = [[x y 1 0 0 0 -X]
       [0 0 0 x y 1 -Y]
        ...
        ...
      ]
x.T = [a0 a1 a2 b0 b1 b2 c3]
```


In [2]:
# TODO: your code goes here

# Task 5:

Find 2D coordinates of a few (five is ok) points that are visible on both photos by hand. Using those coordinates as a ground truth find a projective transformation between right and left photo using results of previous task.

Example result:

```
[[ 8.72296732e-01,  7.44604383e-02, -2.37149377e+02],
 [-8.70238284e-02,  9.30801124e-01,  2.64244035e+00],
 [-1.00727635e-04,  4.18465424e-05,  1.00000000e+00]]
```

In [ ]:
# TODO: your code goes here

# Task 6:

Using the projective transformation you have already found stitch two photos into one. In order to do so:

1. Keep one (in example we keep the right) of images unchanged as base, transform the left image using found transformation.
2. Make a new image and put both images into this image.
3. On area overlaping set point colors as weighted average of corresponding pixel (weight can be based on distance from left and right sides of overlapping areas) 

Results should be similar to results of Task 9.

In [ ]:
# TODO: your code goes here

# Task 7

Now the crazy part. 

So far we have matched points on two photos by hand. It is neither convenient nor useful.
There are many methods of matching points on images. SIFT, SURF and ORB (https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.ORB) are commonly used. However we want to use State Of The Art: SuperGlue.

https://github.com/kciebiera/SuperGluePretrainedNetwork

The repository comes with `./match_pairs.py` script, which allows to easily find matches.
Find matches on your photos using `./match_pairs.py` script from SuperGlue repo.

**This task may be harder than it seems. It works fine on Ubuntu 20.04.**

Plot matches 

<details>
<summary>Example</summary>

![](result_matches.png)

</details>

In [4]:
# Your code goes here

# HINT: there is an example code for reading matching pairs from a file in the repo

# Task 8

Some of the matches are incorrect. Get rid of those matches using RANSAC with projective transformation.

Plot remaining matches 

<details>
<summary>Example</summary>

![](result_matches_filtered.png)

</details>

# Task 9

Make panorama stitching from two photos (like in task 6) using SuperGlue as a matching method.

![](result.png)